<a href="https://colab.research.google.com/github/ccnmagnoo/colabs/blob/main/sec_etl_professional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Resume
Actually there´s 11.700 profesional certificated in GAS and Electricity procedures. But Just 290 of them are women, and [SEC](www.sec.cl) (institution focused in regulate this profesionals), dosen´t focus in this labor. This code is used for extract and filter those women to use in diverse social programs, and visualice their labor and effort in this energy field

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
SOURCE_WEB = 'https://wlhttp.sec.cl/buscadorinstaladores/buscador.do'
PATH_GDRIVE = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality'
eli_source = f'{PATH_GDRIVE}/sec_eli.xls'
gas_source = f'{PATH_GDRIVE}/sec_gas.xls'

#Preparing Dataset

In [38]:
#read gas & dataframe it
gas_df = pd.read_excel(gas_source,header=4)
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              1898 non-null   object
 1   CORREO ELECTRÓNICO  1867 non-null   object
 2   RUT                 1898 non-null   object
 3   DIRECCIÓN           1898 non-null   object
 4   TELÉFONO            902 non-null    object
 5   CELULAR             1844 non-null   object
 6   LICENCIA CLASE      1898 non-null   object
dtypes: object(7)
memory usage: 103.9+ KB


In [39]:
#read electrica & dataframe it
eli_df = pd.read_excel(eli_source,header=4)
eli_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9881 entries, 0 to 9880
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              9881 non-null   object
 1   CORREO ELECTRÓNICO  9611 non-null   object
 2   RUT                 9881 non-null   object
 3   DIRECCIÓN           9881 non-null   object
 4   TELÉFONO            4465 non-null   object
 5   CELULAR             9614 non-null   object
 6   LICENCIA CLASE      9881 non-null   object
dtypes: object(7)
memory usage: 540.5+ KB


In [40]:
#joint gas + eli
all_technicians = pd.concat([eli_df,gas_df])
all_technicians.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              11779 non-null  object
 1   CORREO ELECTRÓNICO  11478 non-null  object
 2   RUT                 11779 non-null  object
 3   DIRECCIÓN           11779 non-null  object
 4   TELÉFONO            5367 non-null   object
 5   CELULAR             11458 non-null  object
 6   LICENCIA CLASE      11779 non-null  object
dtypes: object(7)
memory usage: 736.2+ KB


#Definig Transform Functions

In [41]:
from pandas.io.formats.style_render import non_reducing_slice
from pandas.core.arrays import boolean
#name normalizer
from typing import Union, List
from enum import Enum
import numpy as np


#IMPORTANT : constant used in keras model version 2
CHARS:List[str] = list('aábcdeéfghiíjklmnñoópqrstuúvwxyz')
NAME_BLOCK_SIZE = 20

class Gender(Enum):
  MALE = 'male'
  FEMALE='female'

#normalize name for keras neuronal net consumption
def nameSerializer(unitName:str,size:int=20)->Union[List[float],None]:
  if unitName == None:
    return None

  name_array = list(unitName)
  # [f,i,r,s,t] ->[6/23,8/23,9/23,11/23,20/23] indexation
  descriteName = list(map(lambda char:CHARS.index(char)/len(CHARS),name_array))
  # content into a 20-size box [0.0 ,0.0 ,....0.0]
  box = [0.0]*(size-len(descriteName))
  input = np.concatenate((np.array(box),np.array(descriteName)))
  return list(input.flatten())

#extract just the first name without divisions
class Composer(Enum):
  FIRST = 0
  SECOND = 1

def nameNormalizer(input:str|None,compose:Composer=Composer.FIRST)->Union[str,None]:
  #single word name
  if input == None:
    return None
  #single name checker
  #if len(input.split(' '))==1 and compose == Composer.SECOND:
    #compose = Composer.FIRST

  decomposed_name:str = input.strip().split(' ')[compose.value if len(input.split(' '))>1 else 0].lower()
  #remove al not alpha
  return ''.join([ char for char in decomposed_name if char.isalpha()])

#import ML keras tensorflow model
import keras
from keras.optimizers import get
MODEL = 'keras_genderizer_model_v2.h5'
MODEL_PATH = f'/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/{MODEL}'
model = keras.models.load_model(MODEL_PATH)

#wrapping gender
def getGenderByKeras(unitName:str|None)->Union[Gender,None]:
  if unitName == None:
    return None

  options=[Gender.MALE,Gender.FEMALE]
  input = nameSerializer(unitName,20)

  prediction = model.predict([input])[0]
  category = options[np.argmax(prediction)]
  return category

#for saving process simplified selector with just "o" or "a"
def getGenderSimplified(unitName:str|None)->Gender | None:
  if not unitName:
    return None

  last_char = unitName[-1]
  if last_char == 'o':
    return Gender.MALE
  if last_char == 'a':
    return Gender.FEMALE
  return None

from pandas import DataFrame
WRONG_POSITIVE:DataFrame = pd.read_excel('/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/wrong_positive.xlsx')

def check_is_male_name(unitName:str|None)-> bool:
  if not unitName:
    return False

  return WRONG_POSITIVE['nombre'].str.contains(unitName).any()

#main general function wrapper⭐
def getGender(name:str | None)->Gender | None:
  #checking input is empty
  if name ==  None:
    return None

  #catching wrong positive
  if check_is_male_name(nameNormalizer(name)):
    return Gender.MALE

  #try simplified calculus, for speed 🚀
  firstname_check = getGenderSimplified(nameNormalizer(name))
  if firstname_check:
    return firstname_check.value

  #second try with second name
  second = getGenderSimplified(nameNormalizer(name,Composer.SECOND))
  if second:
    return second.value


  #if simplified dosen´t work applys de keras model
  modeler  = getGenderByKeras(nameNormalizer(name))
  if modeler:
    return modeler.value

  return None

In [42]:
len('Carlos'.split(' '))
getGender('Joshua')

<Gender.MALE: 'male'>

#Transform Dataset & calculating columns

In [43]:
# Transform
## split NOMBRE column
all_technicians[['Apellidos','Nombres']] = all_technicians['NOMBRE'].str.split(',',1,expand=True)
## remove null names
all_technicians['Nombres'] = all_technicians['Nombres'].map(lambda name: name.strip())
all_technicians

<ipython-input-43-939b2ce165c1>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  all_technicians[['Apellidos','Nombres']] = all_technicians['NOMBRE'].str.split(',',1,expand=True)


,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"avenida rica aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel
...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida los puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"mulchen 383, instalador de redes de gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos


##Detecting Genger

In [ ]:
all_technicians['gender'] = all_technicians['Nombres'].map(getGender)

In [45]:
#big filter
from pandas import DataFrame
#result:DataFrame = all_technicians.loc[all_technicians['gender']=='female']
result:DataFrame = all_technicians
result

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,male
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,male
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,male
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"avenida rica aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro,male
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel,male
...,...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida los puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan,Gender.MALE
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"mulchen 383, instalador de redes de gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo,male
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal,male
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos,male


In [52]:
#transform functions

#extract city
def extract_last_col(dir:str)->str|None:
  dir_list = list(reversed(dir.split(',')))
  if len(dir_list)==0:
    return None
  return dir_list[0]


#name case format
import re

def namecase(string:str)->str:
  s_list = re.split('\s+',string.strip())
  for index, word in enumerate(s_list):
    s_list[index] = word[0].upper() + word[1:].lower()
  return ' '.join(s_list)

def setType(input_type:str)->str:
  if input_type == '1 y 4':
    return 'GAS'
  if input_type.isnumeric():
    return 'GAS'
  else:
    return 'ELI'

def getAge(rut:str)->int:
  PENDANT = 3.3363697569700348e-06
  INTERCEPT = 1932.2573852507373
  rut_compose:list[str] = rut.split('-')
  return int(int(rut_compose[0])*PENDANT+INTERCEPT)


  return 0

#Get Territory
from difflib import SequenceMatcher
DPA_PATH = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/DPA2018.xlsx'
DPA = pd.read_excel(DPA_PATH)
from pandas import Series

class Territory(Enum):
  CITY='comuna'
  PROVINCE='prox'
  REGION='region'

def getTerritory (query:str,req:Territory=Territory.CITY,res:Territory=Territory.REGION)->Series:
  #generate a serie with ratio similatities value
  ratio_series:DataFrame = DPA[req.value].apply(lambda req: SequenceMatcher(None,query,req).ratio())
  #find the most simliar ratio an get this index
  most_similar_index = ratio_series.idxmax()
  #extract row index
  result:Series = DPA.iloc[most_similar_index]
  #return value require, like city, region or province
  return result[res.value]


In [47]:
getTerritory('Quillomta',res=Territory.REGION)

'De Valparaíso'

In [53]:
result['Comuna'] = result['DIRECCIÓN'].map(extract_last_col)
result['Comuna'] = result['Comuna'].apply(lambda c:getTerritory(c,res=Territory.CITY))
result['Región'] = result['Comuna'].apply(lambda c:getTerritory(c,res=Territory.REGION))
result['Nac Aprox'] = result['RUT'].apply(getAge)
result['CORREO ELECTRÓNICO'] = result['CORREO ELECTRÓNICO'].str.lower()

result['Categoría'] = result['LICENCIA CLASE'].map(setType)
for field in ['Nombres','Apellidos','DIRECCIÓN','Comuna']:
  result[field] = result[field].map(namecase)

result

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,Apellidos,Nombres,gender,Comuna,Región,Categoría,Edad Aprox
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,male,Las Condes,Metropolitana de Santiago,ELI,1956
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"Colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,male,Conchalí,Metropolitana de Santiago,ELI,1993
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 De Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,male,Los Ángeles,Del Bíobío,ELI,1992
3,"Calderón Barraza , Nabor Alejandro",ncalderon@cjenergy.cl,13977385-3,"Avenida Rica Aventura 11544, Antofagasta",055950899,099591296,A,Calderón Barraza,Nabor Alejandro,male,Antofagasta,De Antofagasta,ELI,1978
4,"Araya Reyes , Jecar Exequiel",jecar.araya@gmail.com,17478701-8,"Pasaje 1 62, Nueva Aurora, Viña Del Mar, Vina ...",NaN,956841025,B,Araya Reyes,Jecar Exequiel,male,Viña Del Mar,De Valparaíso,ELI,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,"Hernández Araya , Alexis Duvan",alexishernandez.araya@gmail.com,19612379-2,"Avenida Los Puelches 921, Chillan",NaN,978706209,1,Hernández Araya,Alexis Duvan,Gender.MALE,Chillán,De Ñuble,GAS,1997
1894,"Rivas Navarrete , Miguel Eduardo",alambreart@gmail.com,17747817-2,"Mulchen 383, Instalador De Redes De Gas ,clase...",413351174,930893107,3,Rivas Navarrete,Miguel Eduardo,male,San Pedro De La Paz,Del Bíobío,GAS,1991
1895,"Quinillao Carvallo , Henrri Aníbal",henrriq@gmail.com,13734890-K,"Malvoa 939, Temuco",NaN,97155737,3,Quinillao Carvallo,Henrri Aníbal,male,Temuco,De La Araucanía,GAS,1978
1896,"Sunkel Ojeda , Juan Carlos",jcsunkel@gmail.com,7641852-7,"Pasaje Navidad 114, Villa Cataluña, Los Ángeles",NaN,978521408,1,Sunkel Ojeda,Juan Carlos,male,Los Ángeles,Del Bíobío,GAS,1957


In [49]:
result_path = f'{PATH_GDRIVE}/sec_result.xlsx'
result.to_excel(result_path)

In [50]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              11779 non-null  object
 1   CORREO ELECTRÓNICO  11478 non-null  object
 2   RUT                 11779 non-null  object
 3   DIRECCIÓN           11779 non-null  object
 4   TELÉFONO            5367 non-null   object
 5   CELULAR             11458 non-null  object
 6   LICENCIA CLASE      11779 non-null  object
 7   Apellidos           11779 non-null  object
 8   Nombres             11779 non-null  object
 9   gender              11779 non-null  object
 10  Comuna              11779 non-null  object
 11  Región              11779 non-null  object
 12  Categoría           11779 non-null  object
dtypes: object(13)
memory usage: 1.3+ MB


In [51]:
result_path_v = f'{PATH_GDRIVE}/sec_result_v.xlsx'
result.loc[result['Región']=='De Valparaíso'].to_excel(result_path_v)